In [1]:
import torchaudio
from transformers import Wav2Vec2FeatureExtractor, Wav2Vec2Processor, Wav2Vec2ForCTC, Wav2Vec2CTCTokenizer
from transformers import Trainer, TrainingArguments
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from datasets import load_metric, Dataset

from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import librosa
from tqdm.notebook import tqdm
import glob
import os
import re
import json

import IPython.display as ipd
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

/home/superceed1/anaconda3/envs/py38torch17/lib/python3.8/site-packages/torchaudio/backend/utils.py:53: UserWarning: "sox" backend is being deprecated. The default backend will be changed to "sox_io" backend in 0.8.0 and "sox" backend will be removed in 0.9.0. Please migrate to "sox_io" backend. Please refer to https://github.com/pytorch/audio/issues/903 for the detail.
  warnings.warn(


# Preprocessing

In [2]:
all_folders = glob.glob('data/*')

In [3]:
data = []
for folder in all_folders[:5]:
    mp3_files = glob.glob(folder + '/sound/*.mp3')
    label_files = glob.glob(folder + '/labels/*.txt')
    
    for mp3, label in zip(mp3_files, label_files):
        text = open(label, 'r').read()
        if text != '':
            data.append({
                'mp3_files': mp3,
                'sentence': text
            })

In [4]:
df = pd.DataFrame(data)
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\'\”\�]'
df['sentence'] = df['sentence'].apply(lambda x: re.sub(chars_to_ignore_regex, '', x).lower() + " ")
df['sentence'] = df['sentence'].apply(lambda x: x.lower())

In [5]:
# Split train and test 
train_df, eval_df = train_test_split(df, test_size=0.2, random_state=2021)

In [6]:
train_data = Dataset.from_pandas(train_df)
eval_data = Dataset.from_pandas(eval_df)

In [7]:
def extract_all_chars(batch):
    all_text = " ".join(batch['sentence'])
    vocab = list(set(all_text))
    return {'vocab': [vocab], 'all_text': [all_text]}

In [8]:
vocab_train = train_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=train_data.column_names)
vocab_eval = eval_data.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=eval_data.column_names)

In [9]:
vocab_list = list(set(vocab_train["vocab"][0]) | set(vocab_eval["vocab"][0]))
vocab_dict = {v:k for k, v in enumerate(vocab_list)}

In [10]:
vocab_dict

{'r': 0,
 '9': 1,
 '6': 2,
 'l': 3,
 'n': 4,
 'm': 5,
 '1': 6,
 'x': 7,
 ' ': 8,
 '7': 9,
 'j': 10,
 's': 11,
 'h': 12,
 'y': 13,
 '2': 14,
 '4': 15,
 'c': 16,
 'g': 17,
 'o': 18,
 '5': 19,
 't': 20,
 'i': 21,
 'q': 22,
 'w': 23,
 'p': 24,
 'k': 25,
 'f': 26,
 'v': 27,
 '3': 28,
 '&': 29,
 'a': 30,
 'u': 31,
 'd': 32,
 '0': 33,
 'z': 34,
 '8': 35,
 'b': 36,
 'e': 37}

In [11]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

40


In [12]:
with open('vocab/vocab.json', 'w') as json_file:
    json.dump(vocab_dict, json_file)

In [13]:
# Create tokenizer
tokenizer = Wav2Vec2CTCTokenizer('vocab/vocab.json', unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

# Feature Extraction

In [14]:
feature_extractor = Wav2Vec2FeatureExtractor(
    feature_size=1, sampling_rate=16000, padding_value=0.0,
    do_normalize=True, return_attention_mask=True
)

In [15]:
processor = Wav2Vec2Processor(
    feature_extractor=feature_extractor, tokenizer=tokenizer,
)

In [16]:
processor.save_pretrained('weights/wave2vec2-malay-processor')

In [17]:
def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = torchaudio.load(batch['mp3_files'])
    batch['speech'] = speech_array[0].numpy()
    batch['sampling_rate'] = sampling_rate
    batch['target_text'] = batch['sentence']
    
    return batch

In [18]:
train_data = train_data.map(speech_file_to_array_fn, remove_columns=train_data.column_names)
eval_data = eval_data.map(speech_file_to_array_fn, remove_columns=eval_data.column_names)

In [19]:
def resample(batch):
    batch['speech'] = librosa.resample(np.asarray(batch['speech']), 44100, 16000)
    batch['sampling_rate'] = 16000
    return batch

In [ ]:
train_data = train_data.map(resample, num_proc=4)
eval_data = eval_data.map(resample, num_proc=4)

In [ ]:
ipd.Audio(data=np.asarray(train_data[0]["speech"]), autoplay=True, rate=16000)

In [ ]:
print("Target text:", train_data[0]["target_text"])
print("Input array shape:", np.asarray(train_data[0]["speech"]).shape)
print("Sampling rate:", train_data[0]["sampling_rate"])

In [ ]:
def prepare_dataset(batch):
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."
    
    batch['input_values'] = processor(batch['speech'], sampling_rate=batch["sampling_rate"][0]).input_values
    
    with processor.as_target_processor():
        batch['labels'] = processor(batch['target_text']).input_ids
        
    return batch

In [ ]:
train_data = train_data.map(prepare_dataset, remove_columns=train_data.column_names, batch_size=8, num_proc=4, batched=True)
eval_data = eval_data.map(prepare_dataset, remove_columns=eval_data.column_names, batch_size=8, num_proc=4, batched=True)

In [ ]:
input_features = [{'input_values': train_data[0]['input_values']}]
label_features = [{'input_ids': train_data[0]['labels']}]

In [ ]:
labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100) 

In [ ]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt"
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt"
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [ ]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    
    return {'wer': wer}

In [ ]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True, max_length_labels=200)
wer_metric = load_metric('wer')

# Model & Train

In [28]:
model = Wav2Vec2ForCTC.from_pretrained(
    'facebook/wav2vec2-large-xlsr-53',
    attention_dropout=0.1,
    hidden_dropout=0.1,
    feat_proj_dropout=0.0,
    mask_time_prob=0.05,
    layerdrop=0.1,
    gradient_checkpointing=True,
    ctc_loss_reduction="mean",
    pad_token_id=processor.tokenizer.pad_token_id,
    vocab_size=len(processor.tokenizer)
)

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-large-xlsr-53 and are newly initialized: ['lm_head.bias', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [29]:
model.freeze_feature_extractor()

In [30]:
training_args = TrainingArguments(
    output_dir='weights/wave2vec2_bm/',
    group_by_length=True,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=30,
    fp16=False, # use fp32 for acc
    save_steps=400,
    eval_steps=400, # Change to print every x steps
    logging_steps=400,
    logging_dir='logs/',
    learning_rate=3e-4,
    warmup_steps=500,
    save_total_limit=2
)

In [31]:
# Train
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_data,
    eval_dataset=eval_data,
    tokenizer=processor.feature_extractor
)

In [32]:
torch.cuda.empty_cache()
trainer.train()

Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
400,7.806300,3.135613,1.000000,29.750700,4.269000
800,3.199300,3.112969,1.000000,29.546700,4.298000
1200,3.137900,3.176500,1.000000,29.563300,4.296000
1600,3.085800,3.158808,1.000000,29.595600,4.291000


TrainOutput(global_step=1890, training_loss=4.1063524034288195, metrics={'train_runtime': 7063.3614, 'train_samples_per_second': 0.268, 'total_flos': 4.611067275695516e+18, 'epoch': 29.99, 'init_mem_cpu_alloc_delta': 60208, 'init_mem_gpu_alloc_delta': 1261919232, 'init_mem_cpu_peaked_delta': 18258, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 500672, 'train_mem_gpu_alloc_delta': 3779377664, 'train_mem_cpu_peaked_delta': 68880945, 'train_mem_gpu_peaked_delta': 1606425600})